In [1]:
# Coding Tidbits for Portfolios
#   ECE478 Financial Signal Processing     Prof. Fred L. Fontaine     Fall 2022
#   Code suggestions here written for clarity and correctness, not efficiency or brevity.

In [2]:
# Import Stuff: You may not need everything here
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import scipy.optimize as optimize
from numpy import random
import yfinance as yfin


In [3]:
# Getting info: test case here is one stock: computing returns and risk-free returns over the period
startdate= datetime(year=2017,month=12,day=31) # last trading day of 2017
enddate= datetime(year=2018,month=12,day=31)
tick= 'AAPL'   # Apple
stock_price= web.DataReader(tick,'yahoo',startdate,enddate)
stock_ret= stock_price.pct_change().dropna()    # Daily stock returns
# Index of stock_ret= all trading days of 2018

fed_data= web.DataReader(['TB3SMFFM','FEDFUNDS'],'fred',startdate,enddate)
fed_data['3MO T-BILL']=fed_data['TB3SMFFM']+fed_data['FEDFUNDS']


In [4]:
# Risk-Free Return
# Let's create a data-frame indexed by the business days in one year
start_date= datetime(year=2017,month=12,day=31)
end_date= datetime(year=2018,month=12,day=31)
stock_price= web.DataReader('AAPL','yahoo',start_date,end_date)
# We want daily risk-free return 
# Data from 'fred': FEDFUNDS= overnight interbank rate, is a "foundational" interest rate in the economy
#    also TB3SMFFM= *spread* between 3-month T-bill and the Fed Funds rate
# Grab both and add together
fed_data= web.DataReader(['TB3SMFFM','FEDFUNDS'],'fred')
# Note: This only goes back to 2017. There are other ways to get older data, or use LIBOR instead.
# In general, 3 month returns on "solid" govt bonds is a reasonably good stand-in for "risk-free interest rate"
# i.e., the basis for evaluating "time value of money"

fed_data['3MO T-BILL']=fed_data['TB3SMFFM']+fed_data['FEDFUNDS']

# The index is Jan-01, Feb-01, etc. through the year. Also, the value is an annual pct
# We first resample to business days, in a way that fills the month with a constant value
# e.g., Jan-01,Jan-02,...,Jan-31 all have same value; then Feb-01 through Feb-28 another value; etc.
# At the end, divide by 100 (pct to numerical) and 252 (# trading days in the year)
fed_3mo_daily= (fed_data['3MO T-BILL'].resample(rule='B').ffill().to_frame())/(100*252)
# Comments about the conversion:
# Normally, when studying bonds, the returns are considered based on calendar days,
#      so the standard daycount convention is   1/360
# But here, we are comparing the return on a risk-free security versus traded stocks. As such
#      the "daily return" should be based on the number of TRADING days, so the daycount convention is  1/252

# We still have a problem. The "resample rule='B'" that in theory only generates "business days" means
#    Saturdays and Sundays are omitted, but it doesn't recognize holidays! e.g., Jan-01 would be in the list
#    unless it was during a weekend! That means the index for 'fed_3mo_daily' is NOT trading days and
#    doesn't match our stock_price data frame. The fix involves extracting a subset of the fed dataframe
#    based on the trading dates in our stock return data frame. [See below]

# Once we fix this, we should subtract the computed risk-free returns from the stock returns to get the `excess returns'
# Since the risk-free return actually varies in time, to better match this to MPT, we should normalize so 
#   the stock return data is based on an equivalent risk-free return of 0, which never changes.
# Your graphs, though, should clearly indicate this (i.e., in (risk,return) plane, label vert axis as 
#   'excess expected return', not 'expected return')
print('fed_3mo_daily: \n',fed_3mo_daily)

print('fed_data: \n',fed_data)

fed_3mo_daily: 
             3MO T-BILL
DATE                  
2017-11-01    0.000049
2017-11-02    0.000049
2017-11-03    0.000049
2017-11-06    0.000049
2017-11-07    0.000049
...                ...
2022-08-26    0.000104
2022-08-29    0.000104
2022-08-30    0.000104
2022-08-31    0.000104
2022-09-01    0.000123

[1262 rows x 1 columns]
fed_data: 
             TB3SMFFM  FEDFUNDS  3MO T-BILL
DATE                                      
2017-11-01      0.07      1.16        1.23
2017-12-01      0.02      1.30        1.32
2018-01-01      0.00      1.41        1.41
2018-02-01      0.15      1.42        1.57
2018-03-01      0.19      1.51        1.70
2018-04-01      0.07      1.69        1.76
2018-05-01      0.16      1.70        1.86
2018-06-01      0.08      1.82        1.90
2018-07-01      0.05      1.91        1.96
2018-08-01      0.12      1.91        2.03
2018-09-01      0.18      1.95        2.13
2018-10-01      0.06      2.19        2.25
2018-11-01      0.13      2.20        2.33
20

In [5]:
# Some pandas dataframes operations
# Given dataframe A and dataframe B where the index set of A is a SUBSET of the index set of B
#    we want to keep only those rows of B that match an index from A
matrix= np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
matrix2= np.array([[10,20,30],[40,50,60],[70,80,90],[100,110,120],[130,140,150]])
A= pd.DataFrame(data=matrix,index=[0,2,4,6],columns=['a','b','c'])
B= pd.DataFrame(data=matrix2,index= [0,2,4,6,10],columns=['aa','bb','cc'])
print('A: \n',A,'\n')
print('B: \n',B,'\n')
B_reduced= B.loc[A.index]
print('B_reduced: \n',B_reduced,'\n')
# Now lets have a single column dataframe (or series) with same indexes, we want to subtract from each column of B_reduced
w= pd.DataFrame(data= [1,2,3,4],index=A.index,columns=['x'])
print('w: \n', w,'\n')
C=B_reduced.subtract(w['x'],axis='rows')
print('C= subtract w from each column of B: \n',C)


A: 
     a   b   c
0   1   2   3
2   4   5   6
4   7   8   9
6  10  11  12 

B: 
      aa   bb   cc
0    10   20   30
2    40   50   60
4    70   80   90
6   100  110  120
10  130  140  150 

B_reduced: 
     aa   bb   cc
0   10   20   30
2   40   50   60
4   70   80   90
6  100  110  120 

w: 
    x
0  1
2  2
4  3
6  4 

C= subtract w from each column of B: 
    aa   bb   cc
0   9   19   29
2  38   48   58
4  67   77   87
6  96  106  116


In [6]:
# Matrix stuff and covariance
M= 100
col= ['a','b','c','d','e']
idx= list(range(1,2*M,2))
stuff= pd.DataFrame(data=np.random.rand(M,5),index=idx,columns=col)
print('Header of a data frame (100 rows, 5 cols)')
print(stuff.head())
print('\n \n')
some_matrix= np.array([[1, 0, 0, 0],[1, 2, 0, 0],[1, 2 ,3, 0],[1, 2, 3, 4],[0,1,0,1]])
col_some_matrix= ['A','B','C','D']
some_dataframe= pd.DataFrame(data=some_matrix,index=stuff.columns,columns=col_some_matrix)
# Multiply dataframe:  (indexA x colA) times (indexB x colB)  where colA=indexB
#  Gives a dataframe  (indexA x colB) :  we can retain LOGIC among the dimensions!
print('Another dataframe: (5rows,4col)')
print(some_dataframe)
stuff2= stuff.dot(some_dataframe)
print('Result of  first data frame x second data frame')
print(stuff2)

# This matrix multiply causes some correlations among the columns
covmat= stuff2.cov()
print('Covariance matrix: note column headings now appear as col/index')
print(covmat)
# Calculate eigenvalues- all should be positive real
eigv = np.linalg.eigvals(covmat)
print('\n \n')
print('Eigenvalues:')
print(eigv)
# Condition number: indicates how numerically unstable handling the matrix is
#   Value >=1, over 100 is "worrisome", approaches infinity as matrix becomes singular
condition_number= eigv.max()/eigv.min()
print('Condition Number')
print(condition_number)
print('\n \n')

# Invert the matrix: we will use "pseudo-inverse" which is a numerically stable approach
#    Note the pinv can even work on rectangular matrices! (It yields "min norm solution to least-squares problem")
covmatinv= np.linalg.pinv(covmat)
print('Inverse cov matrix [note it is a numpy array, not a data frame any more]')
print(covmatinv)

Header of a data frame (100 rows, 5 cols)
          a         b         c         d         e
1  0.683217  0.612796  0.109867  0.883790  0.520361
3  0.471910  0.598212  0.188502  0.400570  0.330384
5  0.026783  0.723312  0.240066  0.857512  0.535328
7  0.056917  0.144580  0.255189  0.878464  0.538181
9  0.563179  0.218100  0.739822  0.986718  0.726528

 

Another dataframe: (5rows,4col)
   A  B  C  D
a  1  0  0  0
b  1  2  0  0
c  1  2  3  0
d  1  2  3  4
e  0  1  0  1
Result of  first data frame x second data frame
            A         B         C         D
1    2.289670  3.733267  2.980972  4.055522
3    1.659194  2.704951  1.767216  1.932664
5    1.847672  4.177107  3.292733  3.965375
7    1.335150  3.094648  3.400961  4.052037
9    2.507818  4.615806  5.179619  4.673399
..        ...       ...       ...       ...
191  1.685297  1.878481  2.510596  0.734517
193  2.025424  3.390047  2.176124  2.488193
195  1.039631  2.677876  3.089930  3.332792
197  2.518540  3.371393  2.669788  3.0

In [7]:
# You will often have to compute values such as transpose(w)*C*w where C is square covar matrix and w is a col vec
# Here is somewhat more general code:
#    A is MxN matrix; x is length-M vector, y is length-N vector. transpose(x)*A*y  is a scalar. Here is how to compute it
#    In this example, stuff is 100x5 dataframe, and x,y are numpy 1-D arrays.
M= 100
col= ['a','b','c','d','e']
idx= list(range(1,2*M,2))
stuff= pd.DataFrame(data=np.random.rand(M,5),index=idx,columns=col)
x= np.random.rand(100,1)
y= np.random.rand(5,1)
x_stuff_y= float(x.transpose().dot(stuff.dot(y)))
# the raw result will be a numpy array [[value]], convert to float to pull it out
print(x_stuff_y)
# This can also be done where x,y,stuff are all dataframes if you want to retain a dataframe type result
# Just make sure index, columns line up properly

91.66721976586734


In [8]:
# Closer look at transpose
aa= stuff.transpose()
aa.head()

,1,3,5,7,9,11,13,15,17,19,...,181,183,185,187,189,191,193,195,197,199
a,0.813126,0.165534,0.037639,0.136141,0.839830,0.766350,0.614798,0.337453,0.964558,0.296573,...,0.055353,0.237392,0.289861,0.908819,0.132148,0.391285,0.484386,0.228249,0.016444,0.543452
b,0.271130,0.251093,0.655892,0.779883,0.935354,0.008963,0.981623,0.718865,0.840251,0.042001,...,0.488037,0.208174,0.622578,0.127320,0.254809,0.938138,0.629994,0.947479,0.402083,0.641804
c,0.825053,0.992615,0.930360,0.268484,0.956049,0.607385,0.587292,0.052941,0.747207,0.088398,...,0.341701,0.056614,0.482364,0.321764,0.443143,0.061871,0.922576,0.918103,0.670994,0.617511
d,0.011158,0.312085,0.866596,0.478468,0.977355,0.985511,0.579251,0.085279,0.470828,0.829180,...,0.549585,0.529599,0.765948,0.922065,0.706672,0.194495,0.090168,0.903941,0.629430,0.785277
e,0.324970,0.511813,0.221726,0.233065,0.226471,0.976607,0.987830,0.267241,0.179594,0.041587,...,0.055208,0.527173,0.137620,0.758079,0.364005,0.918500,0.269333,0.512467,0.664967,0.547432
